# Download

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1aRWqZJakjcX4zMdA-BA-fjjHNclE4THE"})
downloaded.GetContentFile('sudoku_data.csv')

# Solver

In [ ]:
import pandas as pd

class SudokuData:
  def __init__(self, file):
    self.data = pd.read_csv(file)
  
  def _get_index(self, df):
    res = set()
    for index, row in df.iterrows():
      res.add( row['index'] )
    return res
  
  def getRow(self, num):
    row = self.data[ self.data['index'] == num ]['row'].values[0]
    tmp = self.data[ self.data['row'] == row ]
    return self._get_index(tmp)
  
  def getCol(self, num):
    col = self.data[ self.data['index'] == num ]['col'].values[0]
    tmp = self.data[ self.data['col'] == col ]
    return self._get_index(tmp)
  
  def getSquare(self, num):
    box = self.data[ self.data['index'] == num ]['box'].values[0]
    tmp = self.data[ self.data['box'] == box ]
    return self._get_index(tmp)

In [ ]:
class Cell:
  def __init__(self, number):
    self.num     = number
    self.options = set()

    if number == 0:
      self.options = set(range(1, 10))
  
  def remove(self, numbers):
    self.options.difference_update(numbers)
  
  def removeUnique(self, numbers):
    unique = []
    for option in self.options:
      if option not in numbers:
        unique.append(option)
    
    if len(unique) == 1:
      self.num     = unique[0]
      self.options = set()
  
  def check(self):
    if len(self.options) == 1:
      self.num = self.options.pop()
  
  def __str__(self) -> str:
    if self.num == 0:
      return '_'
    else:
      return str(self.num)

In [ ]:
class Sudoku:
  def __init__(self, matrix):
    self.matrix = []
    self.data   = SudokuData('sudoku_data.csv')
    if len(matrix) != 81:
      return
    
    for m in matrix:
      self.matrix.append( Cell(m) )
  
  def _getNumbers(self, index):
    row = set()
    for i in index:
      if self.matrix[i].num != 0:
        row.add( self.matrix[i].num )
    return row
  
  def getSquare(self, num):
    index = self.data.getSquare(num)
    return self._getNumbers(index)
    
  def getRow(self, num):
    index = self.data.getRow(num)
    return self._getNumbers(index)
  
  def getCol(self, num):
    index = self.data.getCol(num)
    return self._getNumbers(index)
  
  def count(self):
    qty = 0
    for m in self.matrix:
      if m.num != 0:
        qty += 1
    return qty
  
  def check(self):
    for i in range(0, len(self.matrix)):
      if self.matrix[i].num != 0:
        continue
      
      rms = self.getSquare(i).union(self.getRow(i)).union(self.getCol(i))
      self.matrix[i].remove( rms )
      self.matrix[i].check()
    return len(self.matrix) - self.count()

  def uniqueOption(self):
    for i in range(0, len(self.matrix)):
      if self.matrix[i].num != 0:
        continue

      indexs = self.data.getSquare(i)
      option = set()
      for index in indexs:
        if self.matrix[index].num != 0 or index == i:
          continue
        
        option = option.union( self.matrix[index].options )
      self.matrix[i].removeUnique( option )

      if self.matrix[i].num != 0:
        continue

      indexs = self.data.getRow(i)
      option = set()
      for index in indexs:
        if self.matrix[index].num != 0 or index == i:
          continue
        
        option = option.union( self.matrix[index].options )
      self.matrix[i].removeUnique( option )

      if self.matrix[i].num != 0:
        continue

      indexs = self.data.getCol(i)
      option = set()
      for index in indexs:
        if self.matrix[index].num != 0 or index == i:
          continue
        
        option = option.union( self.matrix[index].options )
      self.matrix[i].removeUnique( option )
    return len(self.matrix) - self.count()
  
  def __str__(self) -> str:
    qty = 0
    res = ''
    for m in self.matrix:
      res += str(m) + ' '
      qty += 1
      if qty%3 == 0:
        res += '  '
      if qty%9 == 0:
        res = res[:-3]
        res += "\n"
      if qty%27 == 0:
        res += "\n"
    res = res[:-2]
    return res

# Test

## Test 1

In [ ]:
matrix = [ 3,1,0, 6,5,2, 7,0,0,
           6,0,5, 0,8,9, 3,0,0,
           0,0,9, 1,3,4, 5,2,6,
          
           2,3,1, 4,7,0, 0,9,8,
           0,5,0, 2,9,0, 0,3,7,
           9,0,7, 3,1,6, 0,4,0,

           0,0,2, 8,0,1, 9,7,0,
           0,0,0, 9,4,7, 0,5,2,
           0,9,8, 0,0,0, 0,6,1 ]

In [ ]:
sdk = Sudoku(matrix)
print(sdk)

_ _ _   2 6 _   7 _ 1
6 8 _   _ 7 _   _ 9 _
1 9 _   _ _ 4   5 _ _

8 2 _   1 _ _   _ 4 _
_ _ 4   6 _ 2   9 _ _
_ 5 _   _ _ 3   _ 2 8

_ _ 9   3 _ _   _ 7 4
_ 4 _   _ 5 _   _ 3 6
7 _ 3   _ 1 8   _ _ _


In [ ]:
sdk.check()

0

In [ ]:
print(sdk)

4 3 5   2 6 9   7 8 1
6 8 2   5 7 1   4 9 3
1 9 7   8 3 4   5 6 2

8 2 6   1 9 5   3 4 7
3 7 4   6 8 2   9 1 5
9 5 1   7 4 3   6 2 8

5 1 9   3 2 6   8 7 4
2 4 8   9 5 7   1 3 6
7 6 3   4 1 8   2 5 9


## Test 2

In [ ]:
matrix = [ 0,0,0, 2,6,0, 7,0,1,
           6,8,0, 0,7,0, 0,9,0,
           1,9,0, 0,0,4, 5,0,0,
          
           8,2,0, 1,0,0, 0,4,0,
           0,0,4, 6,0,2, 9,0,0,
           0,5,0, 0,0,3, 0,2,8,

           0,0,9, 3,0,0, 0,7,4,
           0,4,0, 0,5,0, 0,3,6,
           7,0,3, 0,1,8, 0,0,0 ]

In [ ]:
sdk = Sudoku(matrix)
print(sdk)

_ _ _   2 6 _   7 _ 1
6 8 _   _ 7 _   _ 9 _
1 9 _   _ _ 4   5 _ _

8 2 _   1 _ _   _ 4 _
_ _ 4   6 _ 2   9 _ _
_ 5 _   _ _ 3   _ 2 8

_ _ 9   3 _ _   _ 7 4
_ 4 _   _ 5 _   _ 3 6
7 _ 3   _ 1 8   _ _ _


In [ ]:
sdk.check()

0

In [ ]:
print(sdk)

4 3 5   2 6 9   7 8 1
6 8 2   5 7 1   4 9 3
1 9 7   8 3 4   5 6 2

8 2 6   1 9 5   3 4 7
3 7 4   6 8 2   9 1 5
9 5 1   7 4 3   6 2 8

5 1 9   3 2 6   8 7 4
2 4 8   9 5 7   1 3 6
7 6 3   4 1 8   2 5 9


# Test 3

In [ ]:
matrix = [ 0,2,0, 6,0,8, 0,0,0,
           5,8,0, 0,0,9, 7,0,0,
           0,0,0, 0,0,4, 0,0,0,
          
           3,7,0, 0,0,0, 5,0,0,
           6,0,0, 0,0,0, 0,0,4,
           0,0,8, 0,0,0, 0,1,3,

           0,0,0, 0,2,0, 0,0,0,
           0,0,9, 8,0,0, 0,3,6,
           0,0,0, 3,0,6, 0,9,0 ]

In [ ]:
sdk = Sudoku(matrix)
print(sdk)

_ 2 _   6 _ 8   _ _ _
5 8 _   _ _ 9   7 _ _
_ _ _   _ _ 4   _ _ _

3 7 _   _ _ _   5 _ _
6 _ _   _ _ _   _ _ 4
_ _ 8   _ _ _   _ 1 3

_ _ _   _ 2 _   _ _ _
_ _ 9   8 _ _   _ 3 6
_ _ _   3 _ 6   _ 9 _


In [ ]:
sdk.check()

53

In [ ]:
sdk.uniqueOption()

53

In [ ]:
print(sdk)

_ 2 _   6 _ 8   _ _ _
5 8 _   _ _ 9   7 _ _
_ _ _   _ _ 4   _ _ _

3 7 _   _ _ _   5 _ _
6 _ _   _ _ 3   _ 7 4
_ _ 8   _ _ _   _ 1 3

_ _ _   9 2 _   _ 7 _
_ _ 9   8 _ _   _ 3 6
_ _ _   3 _ 6   _ 9 _
